In [1]:
from tqdm import tqdm
import cv2
import numpy as np

from utils import list_images
from yunet import YuNet
from sface import SFace

In [12]:
IMAGES_PATH = "/home/rodrigo/Downloads/lfw-deepfunneled"
USE_YUNET = True
THRESHOLD = 0.363  # If we use something above 0.55, the rate of false positive will be below 2%

In [3]:
weights_path = "./sface/bins/face_recognition_sface_2021dec.onnx"
recognizer = SFace(model_path=weights_path, dis_type=0, backend_id=cv2.dnn.DNN_BACKEND_CUDA,
                   target_id=cv2.dnn.DNN_TARGET_CUDA)

face_detector = YuNet()

In [4]:
images_path = list_images(IMAGES_PATH)

hashs = []
for fname in tqdm(list(images_path)):
    image = cv2.imread(fname)
    if USE_YUNET:
        try:  # not all images are going to find some face
            data = face_detector.detect(image)[0][1]
            face_hash = recognizer.infer(image, data)
        except:
            continue
    else:
        face_hash = recognizer.infer(image)
    hashs.append({"hash": cv2.normalize(face_hash, None), "fname": fname})

100%|██████████| 5748/5748 [01:45<00:00, 54.69it/s]


In [5]:
dist_mat = np.zeros((len(hashs), len(hashs)))
for i in tqdm(range(len(hashs))):
    for j in range(len(hashs)):
        if i == j:  # we don't want the same image to be compared with itself
            continue
        dist_mat[i, j] = (hashs[i]["hash"] * hashs[j]["hash"]).sum()  # cosine distance

100%|██████████| 5748/5748 [01:17<00:00, 73.98it/s]


In [6]:
# get best match
x, y = np.unravel_index(dist_mat.argmax(), dist_mat.shape)

print(hashs[x]["fname"])
print(hashs[y]["fname"])
print(x, y, dist_mat.max())

cv2.imshow("image", cv2.imread(hashs[x]["fname"]))
cv2.imshow("image2", cv2.imread(hashs[y]["fname"]))
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/rodrigo/Downloads/lfw-deepfunneled/Eva_Amurri/Eva_Amurri_0001.jpg
/home/rodrigo/Downloads/lfw-deepfunneled/Emmy_Rossum/Emmy_Rossum_0001.jpg
281 4339 0.9704052209854126


In [7]:
# does the same match with the opencv detector, just to compare if the results are the same
recognizer._model.match(hashs[x]["hash"], hashs[y]["hash"])

0.9704051511271246

In [16]:
# how many false positives are there?
rank = {}
matches = {}
for i in range(dist_mat.shape[0]):
    for j in range(dist_mat.shape[1]):
        if dist_mat[i, j] > THRESHOLD + 0.2:
            if i not in rank:
                rank[i] = 1
                matches[i] = [j]
            else:
                rank[i] += 1
                matches[i].append(j)

keys = list(rank.keys())

print("False positives:", len(keys), len(keys) / len(hashs))  # We suppose that there are only picture for each person

False positives: 90 0.015657620041753653


In [9]:
# Which sample has more false positives?
y = dict(sorted(rank.items(), key=lambda x: x[1], reverse=True))
y

{1622: 41,
 3167: 41,
 594: 34,
 1213: 32,
 2420: 28,
 4209: 28,
 5735: 28,
 1067: 27,
 2072: 27,
 3584: 27,
 2750: 26,
 5625: 26,
 751: 25,
 1129: 25,
 1771: 25,
 2441: 25,
 3365: 25,
 4917: 25,
 2027: 24,
 2281: 24,
 2390: 24,
 2408: 24,
 3220: 24,
 5673: 24,
 631: 23,
 1038: 23,
 1045: 23,
 1377: 23,
 3053: 23,
 3228: 23,
 4640: 23,
 241: 22,
 410: 22,
 546: 22,
 559: 22,
 1148: 22,
 1399: 22,
 1577: 22,
 1667: 22,
 1750: 22,
 1897: 22,
 2005: 22,
 2987: 22,
 3006: 22,
 3073: 22,
 3723: 22,
 3927: 22,
 4692: 22,
 5076: 22,
 661: 21,
 1972: 21,
 2539: 21,
 2797: 21,
 3500: 21,
 3608: 21,
 3629: 21,
 4163: 21,
 4167: 21,
 4292: 21,
 4415: 21,
 4623: 21,
 5114: 21,
 5155: 21,
 5230: 21,
 5550: 21,
 105: 20,
 106: 20,
 215: 20,
 308: 20,
 1071: 20,
 1229: 20,
 1702: 20,
 1854: 20,
 2086: 20,
 2675: 20,
 3195: 20,
 3321: 20,
 4492: 20,
 5235: 20,
 5306: 20,
 191: 19,
 491: 19,
 990: 19,
 1470: 19,
 1712: 19,
 1749: 19,
 2043: 19,
 2064: 19,
 2108: 19,
 2418: 19,
 2611: 19,
 2798: 19,
 30

In [10]:
# show it
selected = list(y.keys())[0]
worst = matches[selected]


cv2.imshow("ref", cv2.imread(hashs[selected]["fname"]))
for i in worst:
    # cv2.imshow("face", recognizer._preprocess(faces[i]["face"].image, faces[i]["face"].raw_data))
    cv2.imshow("face", cv2.imread(hashs[i]["fname"]))
    if cv2.waitKey(0) == 27:
        break
cv2.destroyAllWindows()